In [1]:
import pandas as pd
import numpy as np

from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
import optuna


def rmspe(y_true, y_pred):
  """
  Calculate Root Mean Squared Percentage Error (RMSPE)
  """
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  
  # Neglect 0 y_true values
  mask = y_true != 0
  y_true = y_true[mask]
  y_pred = y_pred[mask]

  if len(y_true) == 0:
    return 0  # Return 0 if there are no non-zero y_true values

  # Calculate the percentage errors
  percentage_errors = (y_true - y_pred) / y_true
  
  # Calculate the squared percentage errors
  squared_percentage_errors = percentage_errors ** 2
  
  # Calculate the mean squared percentage error
  mean_squared_percentage_error = np.mean(squared_percentage_errors)
  
  # Calculate the root mean squared percentage error
  root_mean_squared_percentage_error = np.sqrt(mean_squared_percentage_error)
  
  return root_mean_squared_percentage_error

/Users/skalkan/Desktop/rossmann/rossmann_case_study/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
df = pd.read_csv('../data/raw/train.csv', index_col = 'Date')
df = df.drop(columns = ['Id'])
df = df.sort_index()

/var/folders/wy/rd6_hk5s6xx5sy02hxjsz2gr0000gn/T/ipykernel_32154/2532742017.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/train.csv', index_col = 'Date')


In [4]:
# Group the data by 'Store' and count the number of non-null values in each column
store_counts = df.groupby('Store').count()

# Find the store with the most non-null values in all columns
most_available_store = store_counts.sum(axis=1).idxmax()

# Select the data for the most available store
df_selected = df[df['Store'] == most_available_store]

In [5]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Index: 928 entries, 2013-01-01 to 2015-07-17
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Store          928 non-null    int64 
 1   DayOfWeek      928 non-null    int64 
 2   Sales          928 non-null    int64 
 3   Customers      928 non-null    int64 
 4   Open           928 non-null    int64 
 5   Promo          928 non-null    int64 
 6   StateHoliday   928 non-null    object
 7   SchoolHoliday  928 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 65.2+ KB


In [6]:
# Prepare the data for Prophet
df_prophet = df_selected[['Sales']].reset_index()
df_prophet.columns = ['ds', 'y']

# Split the data into train and test sets
train_size = int(len(df_prophet) * 0.8)
train_data = df_prophet[:train_size]
test_data = df_prophet[train_size:]

In [7]:
# Define the objective function for Optuna
def objective(trial):
  # Define the hyperparameters to optimize
  seasonality_mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
  changepoint_prior_scale = trial.suggest_float('changepoint_prior_scale', 0.01, 10.0)
  seasonality_prior_scale = trial.suggest_float('seasonality_prior_scale', 0.01, 10.0)

  # Create and fit the Prophet model with the suggested hyperparameters
  model = Prophet(
      seasonality_mode=seasonality_mode,
      changepoint_prior_scale=changepoint_prior_scale,
      seasonality_prior_scale=seasonality_prior_scale
  )
  model.fit(train_data)

  # Make predictions on the test data
  predictions = model.predict(test_data[['ds']])

  # Evaluate the model
  rmse = sqrt(mean_squared_error(test_data['y'], predictions['yhat']))
  return rmse

# Create a study object
study = optuna.create_study(direction='minimize')

# Optimize the hyperparameters
study.optimize(objective, n_trials=100)




[I 2024-09-10 15:24:22,951] A new study created in memory with name: no-name-ccd16897-f412-460f-baec-bf05ea83e9fd
15:24:23 - cmdstanpy - INFO - Chain [1] start processing
15:24:23 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-09-10 15:24:23,159] Trial 0 finished with value: 1180.7389547361026 and parameters: {'seasonality_mode': 'additive', 'changepoint_prior_scale': 6.280097800661852, 'seasonality_prior_scale': 5.621256280460048}. Best is trial 0 with value: 1180.7389547361026.
15:24:23 - cmdstanpy - INFO - Chain [1] start processing
15:24:23 - cmdstanpy - INFO - Chain [1] done processing
[I 2024-09-10 15:24:23,291] Trial 1 finished with value: 2235.8491922525045 and parameters: {'seasonality_mode': 'multiplicative', 'changepoint_prior_scale': 2.1145829466525687, 'seasonality_prior_scale': 1.6687394043619146}. Best is trial 0 with value: 1180.7389547361026.
15:24:23 - cmdstanpy - INFO - Chain [1] start processing
15:24:23 - cmdstanpy - INFO - Chain [1] done processing
[I 2024

In [8]:
# Get the best hyperparameters
best_params = study.best_params

# Train the model with the best hyperparameters
best_model = Prophet(
    seasonality_mode=best_params['seasonality_mode'],
    changepoint_prior_scale=best_params['changepoint_prior_scale'],
    seasonality_prior_scale=best_params['seasonality_prior_scale']
)
best_model.fit(train_data)


15:25:01 - cmdstanpy - INFO - Chain [1] start processing
15:25:01 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
# Make predictions on the test data using the best model
predictions = best_model.predict(test_data[['ds']])

# Evaluate the best model
rmse = sqrt(mean_squared_error(test_data['y'], predictions['yhat']))
print('Best RMSE:', rmse)

# Evaluate the model with RMSPE
rmspe_score = rmspe(test_data['y'], predictions['yhat'])
print('RMSPE:', rmspe_score)

Best RMSE: 1086.0493185102064
RMSPE: 0.1824658407117263
